### Testing Wind Speed -> Wind Power conversion ###

Based on Hannah's S2S4E and CLEARHEADS scripts. First load bias corrected dcppA-hindcast data from HadGEM3-GC31-MM, then use power law to convert from 10m wind speeds to 100m wind speeds (for now), and then convert into wind power and compare to ERA5.

In [1]:
# cell magic
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Load local modules
import os
import sys
import glob
import importlib

# Import external modules
import numpy as np
import pandas as pd
import xarray as xr
import iris
import matplotlib.pyplot as plt

from tqdm import tqdm

In [2]:
# Import the functions
sys.path.append("/home/users/benhutch/unseen_functions")

import functions_demand as func_dm

import bias_functions as bias_func

import load_wind_functions as lw_func

In [3]:
# import dictionaries
sys.path.append("/home/users/benhutch/unseen_functions")

import unseen_dictionaries as udicts

In [4]:
# set up the global variables - loading for a single country
first_year = 1960
last_year = 1960 # testing loading full model data now
first_month = 1
last_month = 1
init_years = np.arange(1960, 2018 + 1, 1)
variable = "sfcWind"
lead_time = 1
obs_variable = "si100_bc" # use bc 100m winds for obs
country = "United Kingdom"
country_name = "United_Kingdom"

# Set up the path to the obs
# observations from ERA5 containing 10m wind speeds (si10)
# and bias corrected 100m wind speeds (si100_bc)
# obs_path = "/gws/nopw/j04/canari/users/benhutch/ERA5/ERA5_wind_daily_1960_2020.nc"

# Smaller obs path for testing
# process in smaller chunks
obs_path = "/gws/nopw/j04/canari/users/benhutch/ERA5/ERA5_si100_bc_daily_1960_2020.nc"

# Set up the path to the test file
test_file = "/badc/cmip6/data/CMIP6/DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1960-r1i1p1f2/day/sfcWind/gn/files/d20200417/sfcWind_day_HadGEM3-GC31-MM_dcppA-hindcast_s1960-r1i1p1f2_gn_19601101-19601230.nc"

In [5]:
# %%time
# # Load the observations
# test_model_ds = xr.open_dataset(test_file)

# # Load and regrid the observations
# obs_ds = bias_func.load_and_rg_obs(
#     model_ds=test_model_ds,
#     obs_variable=obs_variable,
#     obs_path=obs_path,
#     init_years=init_years,
#     lead_time=lead_time,
#     rg_algo="bilinear",
#     grid_bounds=[-180.0, 180.0, -90.0, 90.0],
#     periodic=True,
#     parallel=False,
# )

In [6]:
# %%time

# # Select the gridbox
# obs_ds = bias_func.select_gridbox(
#     ds=obs_ds,
#     grid=udicts.eu_grid_constrained,
#     calc_mean=False,
# )

In [7]:
# %%time

# # Apply the country mask to the obs data
# obs_ds = lw_func.apply_country_mask(
#     ds=obs_ds,
#     country=country,
#     lon_name="lon",
#     lat_name="lat",
# )

In [8]:
importlib.reload(lw_func)

<module 'load_wind_functions' from '/home/users/benhutch/unseen_functions/load_wind_functions.py'>

In [9]:
# %%time

# # Test the function for creating the wind power data
# cfs, wind_speed_pre , _ = lw_func.create_wind_power_data(
#     ds=obs_ds,
#     country=country_name,
#     ons_ofs="ofs", # test offshore for now
#     onshore_curve_file="/home/users/benhutch/Hannah_model/power_onshore.csv",
#     offshore_curve_file="/home/users/benhutch/Hannah_model/power_offshore.csv",
#     installed_capacities_dir="/home/users/benhutch/Hannah_model/installed_capacities/",
#     lat_name="lat",
#     lon_name="lon",
#     obs_flag=True,
# )

In [10]:
# # create a dataframe of the wind speeds
# obs_ws_cfs_ofs_df = pd.DataFrame({
#     "time": obs_ds["time"].values,
#     "wind_speed": wind_speed_pre,
#     "cfs": cfs,
# })

In [11]:
# # replace the name of wind speed with obs_wind_speed_100m_bc
# obs_ws_cfs_ofs_df.rename(columns={"wind_speed": "obs_wind_speed_100m_bc"}, inplace=True)

# # replace the name of cfs with obs_cfs_ons
# obs_ws_cfs_ofs_df.rename(columns={"cfs": "obs_cfs_ofs"}, inplace=True)

In [12]:
# # set up the save dir
# save_dir = "/gws/nopw/j04/canari/users/benhutch/met_to_energy_dfs/"

# # set up the fname
# fname = f"{country_name}_obs_wind_speed_cfs_ofs.csv"

# # set up the save path
# save_path = os.path.join(save_dir, fname)

# # save the dataframe
# obs_ws_cfs_ofs_df.to_csv(save_path, index=False)

In [13]:
# wind_speed_post_obs = wind_speed_post

In [14]:
# import pandas as pd

# # Convert numpy array to pandas Series
# obs_ds_series = pd.Series(cfs)

# # # Apply a rolling mean (smooth) with a window size of 30
# smoothed_obs_ds = obs_ds_series.rolling(window=360).mean()

# # plot this with a smoothing of 30
# fig, ax = plt.subplots()
# plt.plot(smoothed_obs_ds)

In [15]:
# # %%time

# # # Form the wind power dataframe
# wind_power_df = lw_func.form_wind_power_dataframe(
#     cfs=cfs,
#     ds=obs_ds,
#     country_name=country_name,
# )

In [16]:
# # plot the wind power data with a smoothing of 30
# smooth30 = wind_power_df["United_Kingdom_wind_power"].rolling(window=30).mean()

# # plot this with a smoothing of 30
# fig, ax = plt.subplots()

# plt.plot(smooth30)

### Loading model data ###

Now that we have loaded the obs data, we now want to load the model data and process this into 100m wind speeds before converting to wind power.

In [17]:
importlib.reload(lw_func)

<module 'load_wind_functions' from '/home/users/benhutch/unseen_functions/load_wind_functions.py'>

In [18]:
%%time

# Set up the months of data to load
months = [11] # N test

# set up the base_dir
base_dir = "/work/scratch-nopw2/benhutch/test_nc/"

variable_saved = "__xarray_dataarray_variable__"

# Set up an empty dataframe
combined_df = pd.DataFrame()

# initialize an empty list for the cfs
cfs_list = []

# Loop over the months
for month_idx in tqdm(months):

    # Set up the fname
    fname_month = f"{variable}_bias_correction_HadGEM3-GC31-MM_lead{lead_time}_month{month_idx}_init{first_year}-{last_year}.nc"

    # set up the path
    path = os.path.join(base_dir, fname_month)

    # assert that the path exists
    assert os.path.exists(path), f"Path does not exist: {path}"

    # Select the european domain
    ds = bias_func.select_gridbox(
        ds=xr.open_dataset(path),
        grid=udicts.eu_grid_constrained,
        calc_mean=False,
    )

    # Apply the country mask
    ds = lw_func.apply_country_mask(
        ds=ds,
        country=country,
        lon_name="lon",
        lat_name="lat",
    )

    # # print ds
    # print("ds type: ", type(ds))

    # # print ds type
    # print("type of ds[variable_saved]: ", type(ds[variable_saved]))

    # create wind power data
    totals_mw_pre, totals_mw_rg, wind_regrid, wind_normal = lw_func.create_wind_power_data(
        ds=ds,
        country=country_name,
        ons_ofs="ofs", # test onshore for now
        var_name=variable_saved,
        onshore_curve_file="/home/users/benhutch/Hannah_model/power_onshore.csv",
        offshore_curve_file="/home/users/benhutch/Hannah_model/power_offshore.csv",
        installed_capacities_dir="/home/users/benhutch/Hannah_model/installed_capacities/",
        lat_name="lat",
        lon_name="lon",
        corr_var_name="sfcWind",
        obs_flag=False,
    )

    # # print the shape of CFs    
    # print("cfs shape: ", cfs.shape)

    # # append the cfs to the list
    # cfs_list.append(cfs)

  0%|          | 0/1 [00:00<?, ?it/s]

Found the country!
Pre-country mask
<xarray.Dataset>
Dimensions:                        (lat: 324, lon: 432)
Coordinates:
    lead                           int64 1
    init                           int64 1960
    member                         <U9 'r10i1p1f2'
  * lat                            (lat) float64 -89.72 -89.17 ... 89.17 89.72
  * lon                            (lon) float64 -179.6 -178.8 ... 178.7 179.6
Data variables:
    __xarray_dataarray_variable__  (lat, lon) float64 nan nan nan ... nan nan
[-179.58333336 -178.75000007 -177.91666678 -177.0833335  -176.25000021
 -175.41666693 -174.58333364 -173.75000035 -172.91666707 -172.08333378
 -171.2500005  -170.41666721 -169.58333392 -168.75000064 -167.91666735
 -167.08333406 -166.25000078 -165.41666749 -164.58333421 -163.75000092
 -162.91666763 -162.08333435 -161.25000106 -160.41666778 -159.58333449
 -158.7500012  -157.91666792 -157.08333463 -156.25000134 -155.41666806
 -154.58333477 -153.75000149 -152.9166682  -152.08333491 -15

100%|██████████| 1/1 [00:18<00:00, 18.35s/it]

Installed capacity: 1.4514608 for United_Kingdom
Total MW pre shape: (214, 304)
Total MW regrid shape: (324, 432)
CPU times: user 2.27 s, sys: 3.34 s, total: 5.61 s
Wall time: 18.4 s


In [20]:
totals_mw_pre.max()

0.7224

In [22]:
totals_mw_rg.max()

0.42386845

In [27]:
ic_cube.data.max()

722.4

In [30]:
bc_si100_cube.data.mean()

7.824508751139922

In [21]:
wind_regrid.data.mean()

7.316323010970194

In [22]:
totals_mw.max()

0.7224

In [19]:
print("cfs_list: ", cfs_list)

cfs_list:  [array([[[0.80050884, 0.60930189, 0.61848085, ..., 0.52339014,
         0.37104643, 0.06042947],
        [0.80096073, 0.63605825, 0.62571449, ..., 0.30063697,
         0.26908746, 0.31468432],
        [0.8009047 , 0.64526168, 0.6285922 , ..., 0.18925493,
         0.12954402, 0.08741891],
        ...,
        [0.79666487, 0.65125206, 0.62782693, ..., 0.24594576,
         0.16436792, 0.2365147 ],
        [0.80038536, 0.62852051, 0.62984055, ..., 0.57703675,
         0.83509026, 0.89059864],
        [0.79892864, 0.61640873, 0.62384228, ..., 0.16813298,
         0.50381877, 0.28878024]],

       [[0.59340621, 0.54114222, 0.23368536, ..., 0.31150757,
         0.28026928, 0.48921309],
        [0.60025196, 0.55724975, 0.23055481, ..., 0.60536837,
         0.60401428, 0.76141181],
        [0.59937128, 0.56031219, 0.22675237, ..., 0.24929014,
         0.14227111, 0.04812343],
        ...,
        [0.5968972 , 0.55137135, 0.22602271, ..., 0.25685807,
         0.30798354, 0.43813104],


In [20]:
np.shape(cfs_list)

(5, 59, 10, 30)

In [21]:
# create an array of values for the lead time
lead_time_array_1 = np.arange(1, 30 + 1, 1)
lead_time_array_2 = np.arange(31, 60 + 1, 1)
lead_time_array_3 = np.arange(61, 90 + 1, 1)
lead_time_array_4 = np.arange(91, 120 + 1, 1)
lead_time_array_5 = np.arange(121, 150 + 1, 1)


# combine the arrays with a list
lead_time_list = [lead_time_array_1, lead_time_array_2, lead_time_array_3, lead_time_array_4, lead_time_array_5]

In [22]:
lead_time_list

[array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]),
 array([31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
        48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]),
 array([61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
        78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]),
 array([ 91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120]),
 array([121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
        134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
        147, 148, 149, 150])]

In [23]:
np.shape(cfs_list)

(5, 59, 10, 30)

In [24]:
importlib.reload(lw_func)

<module 'load_wind_functions' from '/home/users/benhutch/unseen_functions/load_wind_functions.py'>

In [25]:
combined_df_test = pd.DataFrame()

In [26]:
for cfs, lead_time_array in zip(cfs_list, lead_time_list):
    print("cfs shape: ", cfs.shape)

    # Form the wind power dataframe
    wp_df = lw_func.form_wind_power_dataframe(
        cfs=cfs,
        ds=ds,
        country_name=country_name,
        obs_flag=False,
        model_fpath=test_file,
        init_years=[1960, 2018],
        leads=lead_time_array,
        ons_ofs="ofs",
    )

    # Append to the combined dataframe
    combined_df_test = pd.concat([combined_df_test, wp_df], ignore_index=True)

cfs shape:  (59, 10, 30)


Looping over init years: 0it [00:00, ?it/s]

Looping over init years: 59it [00:06,  9.46it/s]


cfs shape:  (59, 10, 30)


Looping over init years: 59it [00:06,  9.30it/s]


...

Looping over init years: 59it [00:06,  9.25it/s]


cfs shape:  (59, 10, 30)


Looping over init years: 59it [00:06,  9.31it/s]


cfs shape:  (59, 10, 30)


Looping over init years: 59it [00:06,  9.34it/s]


In [27]:
combined_df_test.lead.unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150])

In [28]:
combined_df_test.head()

,init,lead,member,United_Kingdom_wind_power_cfs_ofs
0,1960,1,1,0.800509
1,1960,1,2,0.800961
2,1960,1,3,0.800905
3,1960,1,4,0.803976
4,1960,1,5,0.802065


In [29]:
# set up the save dir
save_dir = "/gws/nopw/j04/canari/users/benhutch/met_to_energy_dfs/"

# assert that the save_dir exists
assert os.path.exists(save_dir), f"Path does not exist: {save_dir}"

# set up the fname
fname = f"{country_name}_wind_power_df_test_ofs_1960_2018_NDJFM.csv"

# set up the save path
save_path = os.path.join(save_dir, fname)

# if the file does not exist, save it
if not os.path.exists(save_path):
    combined_df_test.to_csv(save_path, index=False)

In [34]:
combined_df_test.head()

,init,lead,member,United_Kingdom_wind_power_cfs_ons
0,1960,1,1,0.682444
1,1960,1,2,0.681283
2,1960,1,3,0.684550
3,1960,1,4,0.687110
4,1960,1,5,0.685891


In [ ]:
# # add a lead column to combined_df
# # one for each unique date
# combined_df["lead"] = np.tile(np.arange(1, 31), len(combined_df) // 30)

In [ ]:
combined_df.head()

In [ ]:
# add lead column to wind_power_df
wind_power_df["lead"] = np.arange(1, len(wind_power_df) + 1)

In [ ]:
# # loop over the different members and plot the wind power data
# fig, ax = plt.subplots()

# for member in combined_df["member"].unique():
#     member_df = combined_df[combined_df["member"] == member]

#     # smooth the member by 10 days centred
#     member_df_smooth = member_df["cfs"].rolling(window=10).mean()

#     plt.plot(member_df["lead"], member_df_smooth, label=member)

# # # quantify a 5%-95% range
# # quantile_5 = member_df_smooth.groupby("lead")["cfs"].quantile(0.05)
# # quantile_95 = member_df_smooth.groupby("lead")["cfs"].quantile(0.95)

# # plt.fill_between(
# #     combined_df["lead"].unique(),
# #     quantile_5,
# #     quantile_95,
# #     alpha=0.2,
# #     color="black",
# #     label="5-95% range",
# # )

# # limit wind power data to 30 days
# wind_power_df_subset = wind_power_df.iloc[:30]

# # # plot the observed wind power data
# obs_smooth = wind_power_df_subset["United_Kingdom_wind_power"].rolling(window=10).mean()

# plt.plot(wind_power_df_subset["lead"], obs_smooth, label="observed", color="black")

# # plot the legend off to the side
# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
wind_speed_post_obs.shape

In [ ]:
wind_speed_post.shape

In [ ]:
# get these into a dataframe
wind_speed_post_obs_df = pd.DataFrame(wind_speed_post_obs)

In [ ]:
# set up a lead column
wind_speed_post_obs_df["lead"] = np.arange(1, len(wind_speed_post_obs_df) + 1)

In [ ]:
# set up column names
wind_speed_post_obs_df.columns = ["obs_100m_wind_speed_bc", "lead"]

In [ ]:
wind_speed_post_obs_df

In [ ]:
wind_speed_dfs = []

# loop over shape of 1th dimension
for i in range(1, wind_speed_pre.shape[1] + 1):

    # get the wind speed
    wind_speed = wind_speed_pre[:, i - 1]

    # print the wind speed shape
    print("wind_speed shape: ", wind_speed.shape)

    # get these into a dataframe
    wind_speed_df = pd.DataFrame({
        "wind_speed": wind_speed,
        "lead": [i] * len(wind_speed),
        "member": np.arange(1, wind_speed.shape[0] + 1),
    })

    # append to the list
    wind_speed_dfs.append(wind_speed_df)

# concatenate the dataframes
wind_speed_df = pd.concat(wind_speed_dfs, ignore_index=True)

In [ ]:
wind_speed_df

In [ ]:
# set up a figure
fig, ax = plt.subplots()

# # subset df to 30 days
# wind_speed_pre_obs_df_subset = wind_speed_pre_obs_df.iloc[:30]

# take a rolling mean
obs_smooth = wind_speed_pre_obs_df_subset["obs_10m_wind_speed"].rolling(window=10).mean()

# plot the observed wind speed
plt.plot(wind_speed_pre_obs_df_subset["lead"], obs_smooth, label="observed", color="black")

# loop over the different members and plot the wind power data
for member in wind_speed_df["member"].unique():
    member_df = wind_speed_df[wind_speed_df["member"] == member]

    # smooth the member by 10 days centred
    member_df_smooth = member_df["wind_speed"].rolling(window=10).mean()

    plt.plot(member_df["lead"], member_df_smooth, label=member)

# plot the legend off to the side
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# add a ylabel
plt.ylabel("10m wind speed (m/s)")